## Imports

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from tqdm.auto import tqdm

## Read Parameters

In [2]:
folder = os.path.join(
    os.environ["SYNC"], r"MotionData\data\2nd-Development", "parsedData"
)

event_name = {
    "0-SLW": 1,
    "1-MLW": 2,
    "2-FLW": 3,
    "3-RD": 4,
    "4-SD": 5,
    "5-sit": 6,
    "6-stand": 7,
    "7-RA": 8,
    "8-SA": 9,
}

# Read and mark the Signal
X3s = []
ys = []

for name, label in event_name.items():
    with open(os.path.join(folder, f"{name}.npy"), "rb") as f:
        mat = np.load(f)
    X3s.append(mat)
    ys.append(np.zeros(mat.shape[0]) + label)

X3 = np.concatenate(X3s)
y = np.concatenate(ys)

# Simple Report
X3.shape, y.shape

((31625, 20, 6), (31625,))

## Take an Example

In [3]:
pd.DataFrame(X3[0])

,0,1,2,3,4,5
0,-0.46,-0.35,-1.46,-48.23,196.47,44.74
1,-0.47,-0.21,-1.49,-60.67,180.91,52.67
2,-0.46,-0.04,-1.51,-26.38,179.34,56.34
3,-0.36,0.06,-1.45,24.03,177.45,47.34
4,-0.12,-0.01,-1.32,49.87,171.94,39.17
5,0.13,-0.13,-1.26,63.96,161.90,28.58
6,0.28,-0.25,-1.28,67.75,144.61,23.74
7,0.29,-0.28,-1.28,61.36,112.47,21.50
8,0.31,-0.27,-1.23,49.75,78.09,19.36
9,0.29,-0.18,-1.19,36.56,49.82,18.97


## Compute the Trace

In [4]:
def scale(v, s=1):
    """ Scale the vector of y \in R3,
        s is the  scaler,
        It computes the normalized vector of y is s = 1 as default.
    """
    return s * np.array(v) / np.linalg.norm(v)


def rotate(r, n, a):
    """ Compute the rotated vector of r \in R3,
        n \in R3 is the rotate axis,
        a \in R is the radius.
    """
    d = np.cross(n, r)

    if all(d == 0):
        return r

    o = scale(scale(r, np.cos(a)) + scale(d, np.sin(a)))
    return o


class Point(object):
    """ A point has its position and x-, y- and z- normalized coordinates """

    def __init__(self, p=[0, 0, 0], nx=[1, 0, 0], ny=[0, 1, 0], nz=[0, 0, 1]):
        """
        Initialize the point,
        - p: The origin of the point;
        - nx: The normed x axis;
        - ny: The normed y axis;
        - nz: The normed z axis.
        """
        self.p = np.array(p).astype(np.float32)
        self.nx = scale(nx)
        self.ny = scale(ny)
        self.nz = scale(nz)

    def translate(self, dp):
        """ Translate the point by p \in R3 """
        self.p += dp
        pass

    def rotate(self, ax, ay, az):
        """ The point rotates around its origin, by 3 axis at ax, ay and az radius """
        # self.nx = rotate(self.nx, self.nx, ax)
        self.ny = rotate(self.ny, self.nx, ax)
        self.nz = rotate(self.nz, self.nx, ax)

        self.nx = rotate(self.nx, self.ny, ay)
        # self.ny = rotate(self.ny, self.ny, ay)
        self.nz = rotate(self.nz, self.ny, ay)

        self.nx = rotate(self.nx, self.nz, az)
        self.ny = rotate(self.ny, self.nz, az)
        # self.nz = rotate(self.nz, self.nz, az)
        pass

    def snip(self):
        """ Get a quick snip of the point on the current state"""
        return [self.p.copy(), self.nx, self.ny, self.nz]


class MotionTrace(object):
    def __init__(self):
        self.point = Point()
        self.trace = [self.point.snip()]

    def forward(self, dp=(0, 0, 0), ax=0, ay=0, az=0):
        self.point.translate(dp)
        self.point.rotate(ax, ay, az)
        self.trace.append(self.point.snip())
        pass

In [5]:
mt = MotionTrace()

for d in X3[0]:
    mt.forward(dp=d[3:], ax=d[0], ay=d[1], az=d[2])

mt.trace

[[array([0., 0., 0.], dtype=float32),
  array([1., 0., 0.]),
  array([0., 1., 0.]),
  array([0., 0., 1.])],
 [array([-48.23, 196.47,  44.74], dtype=float32),
  array([ 0.10386623, -0.87372632,  0.47519903]),
  array([0.93361282, 0.25037175, 0.25628322]),
  array([-0.34289781,  0.41703274,  0.84172727])],
 [array([-108.9 ,  377.38,   97.41], dtype=float32),
  array([-0.97428234, -0.09504764,  0.20430338]),
  array([ 0.20529835, -0.74815549,  0.63096431]),
  array([0.09287903, 0.65668053, 0.7484278 ])],
 [array([-135.28   ,  556.72003,  153.75   ], dtype=float32),
  array([-0.20118624,  0.95499381, -0.21797   ]),
  array([-0.95607274, -0.1430118 ,  0.25587601]),
  array([0.21318772, 0.25987391, 0.94181556])],
 [array([-111.25   ,  734.17004,  201.09   ], dtype=float32),
  array([0.93961049, 0.33722766, 0.05839199]),
  array([-0.30806299,  0.9076896 , -0.28495048]),
  array([-0.14909498,  0.24975405,  0.95676204])],
 [array([-61.38   , 906.11005, 240.26   ], dtype=float32),
  array([ 0.51

In [28]:
p = mt.trace[0]

data = []

for p in tqdm(mt.trace):
    origin = p[0]
    for axis, color in zip(p[1:], ['red', 'green', 'blue']):
        data.append(
            go.Scatter3d(
                x=[origin[0], origin[0] + axis[0] * 100],
                y=[origin[1], origin[1] + axis[1] * 100],
                z=[origin[2], origin[2] + axis[2] * 100],
                marker={'size': 2, 'color': color},
                line={"color": color, "width": 2},
            )
        )

# data = [
#     go.Scatter3d(x=[0, 1], y=[0, 0], z=[0, 0], line={"color": "darkblue", "width": 2})
# ]
fig = go.Figure(data=data)
fig.show()

  0%|          | 0/21 [00:00<?, ?it/s]

In [15]:
data

[Scatter3d({
     'line': {'color': 'red', 'width': 2}, 'x': [0.0, 1.0], 'y': [0.0, 0.0], 'z': [0.0, 0.0]
 }),
 Scatter3d({
     'line': {'color': 'green', 'width': 2}, 'x': [0.0, 0.0], 'y': [0.0, 1.0], 'z': [0.0, 0.0]
 }),
 Scatter3d({
     'line': {'color': 'blue', 'width': 2}, 'x': [0.0, 0.0], 'y': [0.0, 0.0], 'z': [0.0, 1.0]
 }),
 Scatter3d({
     'line': {'color': 'red', 'width': 2},
     'x': [-48.22999954223633, 0.10386623402851615],
     'y': [196.47000122070312, -0.8737263207545458],
     'z': [44.7400016784668, 0.47519903393152874]
 }),
 Scatter3d({
     'line': {'color': 'green', 'width': 2},
     'x': [-48.22999954223633, 0.9336128207511815],
     'y': [196.47000122070312, 0.25037174798192097],
     'z': [44.7400016784668, 0.2562832197813579]
 }),
 Scatter3d({
     'line': {'color': 'blue', 'width': 2},
     'x': [-48.22999954223633, -0.3428978074554514],
     'y': [196.47000122070312, 0.4170327376036587],
     'z': [44.7400016784668, 0.8417272654542219]
 }),
 Scatter3d({
 